# ***Dependencies Installation***

In [ ]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 466.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [ ]:
pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

# ***Importing Libraries and Downloading Dataset***

In [ ]:
import yaml
from roboflow import Roboflow
import os
from pprint import pprint
import cv2
import random
import matplotlib.pyplot as plt
import glob



In [ ]:
rf = Roboflow(api_key="yHaUGg4T6jmlN51vfxAp")
project = rf.workspace("omar-z9f8z").project("olive-axotr")
version = project.version(3)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Olive-3 in yolov11:: 100%|██████████| 29712/29712 [00:06<00:00, 4531.50it/s]


# ***Dataset Exploration***

In [ ]:

# ✅ Corrected YAML path: wrap the path string in quotes
yaml_path = os.path.join("/content/Olive-3", "data.yaml")

# Load and print YAML
with open(yaml_path, "r") as f:
    data_yaml = yaml.safe_load(f)

print("YAML Content:")
pprint(data_yaml)


YAML Content:
{'names': ['olive'],
 'nc': 1,
 'roboflow': {'license': 'CC BY 4.0',
              'project': 'olive-axotr',
              'url': 'https://universe.roboflow.com/omar-z9f8z/olive-axotr/dataset/3',
              'version': 3,
              'workspace': 'omar-z9f8z'},
 'test': '../test/images',
 'train': '../train/images',
 'val': '../valid/images'}


In [ ]:
def count_images_and_labels(base_dir="/content/Olive-3"):
    for split in ['train', 'valid', 'test']:
        image_dir = os.path.join(base_dir, split, 'images')
        label_dir = os.path.join(base_dir, split, 'labels')

        image_count = len(os.listdir(image_dir))
        label_count = len(os.listdir(label_dir))

        print(f" {split.upper()} set: {image_count} images, {label_count} labels")

count_images_and_labels()


 TRAIN set: 12360 images, 12360 labels
 VALID set: 1773 images, 1773 labels
 TEST set: 717 images, 717 labels


In [ ]:

def load_labels_txt(label_path):
    with open(label_path, 'r') as f:
        return [list(map(float, line.strip().split())) for line in f.readlines()]

def show_images_with_boxes(split="train", base_dir="/content/Olive-3", num_samples=5):
    image_dir = os.path.join(base_dir, split, "images")
    label_dir = os.path.join(base_dir, split, "labels")

    image_files = os.listdir(image_dir)
    selected = random.sample(image_files, num_samples)

    for file_name in selected:
        image_path = os.path.join(image_dir, file_name)
        label_path = os.path.join(label_dir, file_name.replace(".jpg", ".txt").replace(".png", ".txt"))

        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape

        labels = load_labels_txt(label_path)

        for cls, x, y, bw, bh in labels:
            # Convert normalized to pixel coordinates
            x1 = int((x - bw / 2) * w)
            y1 = int((y - bh / 2) * h)
            x2 = int((x + bw / 2) * w)
            y2 = int((y + bh / 2) * h)

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, "olive", (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        plt.figure(figsize=(6,6))
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"{split} sample")
        plt.show()

show_images_with_boxes("train", num_samples=4)


ValueError: too many values to unpack (expected 5)

# ***Object Detection : YOLO11m***

**version 1**

In [ ]:
from ultralytics import YOLO
model = YOLO('yolo11m.pt')

# Train with optimized settings
model.train(
    data='/content/Olive-3/data.yaml',
    epochs=20,
    imgsz=640,
    batch=8,
    patience=10,
    name='olivedetect_yolov11m',
    lr0=0.002,
    lrf=0.01,
    warmup_epochs=5,
    weight_decay=0.0005,
    optimizer='AdamW',

    # Loss weights
    box=5.0,
    cls=0.3,
    dfl=2.0,

    mosaic=1.0,
    mixup=0.15,
    hsv_h=0.015,
    hsv_s=0.6,
    hsv_v=0.4,
    degrees=0.0,
    translate=0.2,
    scale=0.5,
    shear=0.0,
    fliplr=0.5,
    flipud=0.0,
    perspective=0.0,
    save_period=5,
    verbose=True
)

100%|██████████| 38.8M/38.8M [00:00<00:00, 194MB/s]


Ultralytics 8.3.115 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/content/Olive-3/data.yaml, epochs=20, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=8, project=None, name=olivedetect_yolov11m, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True

100%|██████████| 755k/755k [00:00<00:00, 22.3MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics

 13                  -1  1   1642496  ultralytics.nn.modules.block.C3k2            [1024, 512, 1, True]          
 14                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 15             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 16                  -1  1    542720  ultralytics.nn.modules.block.C3k2            [1024, 256, 1, True]          
 17                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
 18            [-1, 13]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 19                  -1  1   1511424  ultralytics.nn.modules.block.C3k2            [768, 512, 1, True]           
 20                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
 21            [-1, 10]  1         0  ultralytics.nn.modules.conv.Concat           [1]  

...

                   all       1773      21223      0.731      0.738      0.774      0.524



20 epochs completed in 2.796 hours.
Optimizer stripped from runs/detect/olivedetect_yolov11m/weights/last.pt, 40.5MB
Optimizer stripped from runs/detect/olivedetect_yolov11m/weights/best.pt, 40.5MB

Validating runs/detect/olivedetect_yolov11m/weights/best.pt...
Ultralytics 8.3.115 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 111/111 [00:29<00:00,  3.73it/s]


                   all       1773      21223       0.73      0.738      0.774      0.524
Speed: 0.2ms preprocess, 10.2ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/olivedetect_yolov11m


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f073be887d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
last_ckpt="/content/runs/detect/olivedetect_yolov11m/weights/last.pt"
model.export(format="onnx")
model.save("Saved_11.onnx")

Ultralytics 8.3.115 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.00GHz)
YOLO11m summary (fused): 125 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs

PyTorch: starting from 'runs/detect/olivedetect_yolov11m/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (38.6 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 225.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 272.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.8/280.8 MB 236.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 149.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 200.6 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 19.9s, installed 3 packages: ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu']
WARNING ⚠️ requirements: Restart runtime

**version 2:**

In [ ]:
from ultralytics import YOLO
oliveDetect_11_model3= YOLO('yolo11m.pt')
training_results3__yolo11 = oliveDetect_11_model3.train(
    data='/content/Olive-3/data.yaml',
    epochs=20,
    imgsz=640,
    batch=8,
    patience=21,
    name='olive_detection_v11',
    box=5.0,
    cls=0.3,
    dfl=2.5,
    save_period=5,
    # Minimal augmentations
    hsv_h=0.01,
    hsv_s=0.5,
    hsv_v=0.3,
    fliplr=0.0,
    flipud=0.0,
    translate=0.0,
    scale=0.0
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 38.8M/38.8M [00:00<00:00, 90.4MB/s]


Ultralytics 8.3.118 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/content/Olive-3/data.yaml, epochs=20, time=None, patience=21, batch=8, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=8, project=None, name=olive_detection_v11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

100%|██████████| 755k/755k [00:00<00:00, 41.5MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics

...

      20/20      7.12G     0.5597     0.3587      1.612         56        640: 100%|██████████| 1545/1545 [07:59<00:00,  3.22it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 111/111 [00:30<00:00,  3.67it/s]

                   all       1773      21223      0.747      0.707      0.741      0.499



20 epochs completed in 2.933 hours.
Optimizer stripped from runs/detect/olive_detection_v11/weights/last.pt, 40.5MB
Optimizer stripped from runs/detect/olive_detection_v11/weights/best.pt, 40.5MB

Validating runs/detect/olive_detection_v11/weights/best.pt...
Ultralytics 8.3.118 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 111/111 [00:32<00:00,  3.41it/s]


                   all       1773      21223      0.724      0.722      0.755      0.512
Speed: 0.2ms preprocess, 11.0ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/olive_detection_v11


In [ ]:
oliveDetect_11_model3.save("oliveDetection3_YOLO11.pt")

version3:

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 38.8M/38.8M [00:00<00:00, 287MB/s]


Ultralytics 8.3.119 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/content/Olive-3/data.yaml, epochs=30, time=None, patience=31, batch=8, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=8, project=None, name=olive_detection_v11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

100%|██████████| 755k/755k [00:00<00:00, 24.5MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics

...

      30/30      6.37G     0.5199     0.3333      1.558         60        640: 100%|██████████| 1545/1545 [07:37<00:00,  3.38it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 111/111 [00:28<00:00,  3.96it/s]

                   all       1773      21223      0.765      0.713       0.76      0.518



30 epochs completed in 4.146 hours.
Optimizer stripped from runs/detect/olive_detection_v11/weights/last.pt, 40.5MB
Optimizer stripped from runs/detect/olive_detection_v11/weights/best.pt, 40.5MB

Validating runs/detect/olive_detection_v11/weights/best.pt...
Ultralytics 8.3.119 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 111/111 [00:29<00:00,  3.70it/s]


                   all       1773      21223      0.739      0.725      0.774      0.533
Speed: 0.2ms preprocess, 10.2ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/olive_detection_v11


In [ ]:
oliveDetect_11_model4.save("oliveDetection4_YOLO11.pt")

In [ ]:
from ultralytics import YOLO
oliveDetect_11_model4= YOLO('yolo11m.pt')
training_results4__yolo11 = oliveDetect_11_model4.train(
    data='/content/Olive-3/data.yaml',
    epochs=35,
    imgsz=640,
    batch=8,
    patience=36,
    name='olive_detection_v11',
    box=5.0,
    cls=0.3,
    dfl=2.5,
    save_period=5,
    # Minimal augmentations
    hsv_h=0.01,
    hsv_s=0.5,
    hsv_v=0.3,
    fliplr=0.0,
    flipud=0.0,
    translate=0.0,
    scale=0.0
)

Ultralytics 8.3.119 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/content/Olive-3/data.yaml, epochs=35, time=None, patience=36, batch=8, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=8, project=None, name=olive_detection_v112, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True,

train: Scanning /content/Olive-3/train/labels.cache... 12360 images, 5 backgrounds, 0 corrupt: 100%|██████████| 12360/12360 [00:00<?, ?it/s]

train: /content/Olive-3/train/images/IMG_20221105_094810_jpg.rf.97ce47c2d8a19d7b5015816af499f942.jpg: 3 duplicate labels removed
train: /content/Olive-3/train/images/IMG_20221105_094810_jpg.rf.ac6a65b6b3fc89eca93428f3203f5ac3.jpg: 3 duplicate labels removed
train: /content/Olive-3/train/images/IMG_20221105_131058_jpg.rf.2c8f2b289fb1efac3855971559a8bcb1.jpg: 1 duplicate labels removed
train: /content/Olive-3/train/images/IMG_20221105_131058_jpg.rf.4a3caaad66bfdc06cc79149c3c17d5a5.jpg: 1 duplicate labels removed
train: /content/Olive-3/train/images/IMG_20221105_131454_jpg.rf.99cf935ee1da5b13ee672ecf0ad88bda.jpg: 1 duplicate labels removed
train: /content/Olive-3/train/images/IMG_20221105_131454_jpg.rf.fe58f197192cf906d05b9e46d861dfab.jpg: 1 duplicate labels removed
train: /content/Olive-3/train/images/IMG_20221112_102138_jpg.rf.109898422b1bf4e82099adf2ceef176e.jpg: 1 duplicate labels removed
train: /content/Olive-3/train/images/IMG_20221112_102138_jpg.rf.8e99469ddf7179b4897c2e1ba51c1b63.

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 722.0±472.0 MB/s, size: 71.2 KB)


val: Scanning /content/Olive-3/valid/labels.cache... 1773 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1773/1773 [00:00<?, ?it/s]

val: /content/Olive-3/valid/images/IMG_20221105_131506_jpg.rf.4a35c52b8895a7a2ff79ba09b120f571.jpg: 3 duplicate labels removed
val: /content/Olive-3/valid/images/IMG_20221112_114559_jpg.rf.63df1e158be744a9ebee708ceb16d85e.jpg: 3 duplicate labels removed
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 907, len(boxes) = 21223. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


Plotting labels to runs/detect/olive_detection_v112/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.0005), 112 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/olive_detection_v112
Starting training for 35 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/35      4.65G      1.129      1.309        2.5         98        640:   6%|▌         | 86/1545 [00:28<07:41,  3.16it/s]